In [6]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.DnCNN as DnCNN, models.ResNet as ResNet, models.RFDN as RFDN, models.ResNet_ED as ResNetED, models.DRLN as DRLN, models.pix2pix as pix2pix
from utils.param import param_check, seed_everything

model_list = [
            #     'DnCNN', 'ResNet18', 
            # #   'ResNet34', 'ResNet50', 'ResNet101','ResNet152', 
            #   'RFDN', 'ResNetED', 
            # 'DRLN', 
            'pix2pix']
models = {'DnCNN': DnCNN.DnCNN(), 
          'ResNet18': ResNet.ResNet18(), 
          'ResNet34': ResNet.ResNet34(), 
          'ResNet50': ResNet.ResNet50(), 
          'ResNet101': ResNet.ResNet101(), 
          'ResNet152': ResNet.ResNet152(), 
          'RFDN': RFDN.RFDN(),
          'ResNetED' : ResNetED.ResNet18(),
          'DRLN' : DRLN.DRLN(),
          }

def param_print(model):
    param_check(model)
    param_check(model, True)
    print(summary(model, (3, 128, 128)))

device = torch.device('cuda' if torch.cuda.is_available() else 'mps:0' if torch.backends.mps.is_available() else 'cpu')
for m in model_list:   
    print('\n\n',m,' 모델은 다음과 같다.')
    if m == 'pix2pix':
        G = pix2pix.Generator()
        D = pix2pix.Discriminator()
        print('총 : ',param_check(G) + param_check(D))
        print('총 : ',param_check(G, True) + param_check(D, True))
        print(summary(G, (3, 256, 256)))
        print(summary(D, (6, 256, 256)))
    else:
        param_print(models[m])
    # param_check(models[m])
    # param_check(models[m], True)
    # print(summary(models[m], (3, 128, 128)))



 pix2pix  모델은 다음과 같다.
모든 parameter 개수기준임
Number of parameters: 54413958
!!!!!!10M 1천만보다 44413958개 초과했음!!!!!!
모든 parameter 개수기준임
Number of parameters: 2768642
10M 1천만보다 7231358개 여유 있음...
총 :  57182600
grad인 parameter 개수기준임
Number of parameters: 54413958
!!!!!!10M 1천만보다 44413958개 초과했음!!!!!!
grad인 parameter 개수기준임
Number of parameters: 2768642
10M 1천만보다 7231358개 여유 있음...
총 :  57182600
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           3,072
         LeakyReLU-2         [-1, 64, 128, 128]               0
            Conv2d-3          [-1, 128, 64, 64]         131,072
       BatchNorm2d-4          [-1, 128, 64, 64]             256
         LeakyReLU-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 256, 32, 32]         524,288
       BatchNorm2d-7          [-1, 256, 32, 32]             512
         LeakyReLU-8          [-1, 25